<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import os
import pandas as pd
import numpy as np
import re
import string
import requests
import nltk
from nltk import pos_tag
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score, confusion_matrix
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from lightgbm import LGBMClassifier
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
tqdm.pandas()

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    data= pd.read_csv(pth1, parse_dates=[0], index_col=[0])
elif os.path.exists(pth2):
    data= pd.read_csv(pth2, parse_dates=[0], index_col=[0])
else:
    print('Something is wrong')
    
display(data.head())
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [3]:
# проверим наличие дубликатов
data.duplicated().sum()

0

In [4]:
# найдем уникальные значения для столбца toxic
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [5]:
# очистим и лемматизируем текст
def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.split())

In [6]:
def lemmatize(text):
    m = WordNetLemmatizer()
    txt_list = nltk.word_tokenize(text)
    pos_tags = pos_tag(txt_list)
    lemmatized_words = []
    for word, tag in pos_tags:
        # Преобразуем тег NLTK в формат, который понимает WordNetLemmatizer
        if tag.startswith('J'):
            pos = 'a'  # прилагательные
        elif tag.startswith('V'):
            pos = 'v'  # глаголы
        elif tag.startswith('N'):
            pos = 'n'  # существительные
        elif tag.startswith('R'):
            pos = 'r'  # наречия
        else:
            pos = 'n'  # по умолчанию используем существительное
        lemmatized_words.append(m.lemmatize(word, pos))
    return ' '.join(lemmatized_words)

In [7]:
# тестируем добавление POS тегов
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)
print(df_my['text'].apply(lemmatize))

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    The striped bat be hang on their foot for best
1         you should be ashamed of yourself go work
Name: text, dtype: object


In [8]:
data['lemm_text'] = data['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

  0%|          | 0/159292 [00:00<?, ?it/s]

In [9]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not try to edit war It 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


In [10]:
# подготовим выборки
train, test = train_test_split(data, test_size=0.1, random_state=12345)

target_train = train['toxic']
features_train = train['lemm_text']

target_test = test['toxic']
features_test = test['lemm_text']

In [11]:
# cоздадим матрицу cо значениями TF-IDF по корпусу и укажим стоп-слова 
stopwords_ = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords_)
tf_idf = count_tf_idf.fit_transform(tqdm(features_train))
tf_idf_test = count_tf_idf.transform(features_test)

  0%|          | 0/143362 [00:00<?, ?it/s]

Выводы:

В датафрейме data 159292 записей, названия столбцов адекватны содержащимся в них данным. Было проверено, нет ли дубликатов. Выявлено, что в датафрейме 143106 нетоксичных комментарией и 16186 токсичных. Текст был очищен и лемматизирован с помощью WordNetLemmatizer, был создан столбец с лемматизированным текстом. Были подготовлены тестовые и тренировочные выборки для обучения. Создана матрица cо значениями TF-IDF и указаны стоп-слова. Данные готовы к обучению.

## Обучение

Модель LogisticRegression

In [12]:
# разделим обучающие данные на обучающую и валидационную выборки
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    tf_idf, target_train, test_size=0.2, random_state=12345
)

# функция для оценки модели
def evaluate_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)  
    predicted = model.predict(X_val)  
    return f1_score(y_val, predicted)  

# поиск лучшего значения C
best_score = 0
best_c = None

for c in range(1, 15):
    model = LogisticRegression(
        class_weight='balanced',
        random_state=12345,
        max_iter=100,
        solver='liblinear',
        C=c
    )

    # оценка модели на разделенной валидационной выборке
    score = evaluate_model(model, X_train_split, y_train_split, X_val_split, y_val_split)

    print("Значение C:", c)
    print("F1_score на валидационных данных:", score)
    print("")

    if score > best_score:
        best_score = score
        best_c = c

print("Лучшее значение C:", best_c)
print("Лучший F1_score на валидационных данных:", best_score)

Значение C: 1
F1_score на валидационных данных: 0.7400510587175252

Значение C: 2
F1_score на валидационных данных: 0.7450800915331808

Значение C: 3
F1_score на валидационных данных: 0.7489966038900896

Значение C: 4
F1_score на валидационных данных: 0.7511693171188026

Значение C: 5
F1_score на валидационных данных: 0.7533364735437273

Значение C: 6
F1_score на валидационных данных: 0.7558856059409069

Значение C: 7
F1_score на валидационных данных: 0.756585211044113

Значение C: 8
F1_score на валидационных данных: 0.7552514322087841

Значение C: 9
F1_score на валидационных данных: 0.7556618819776715

Значение C: 10
F1_score на валидационных данных: 0.754444978375781

Значение C: 11
F1_score на валидационных данных: 0.7546140266409886

Значение C: 12
F1_score на валидационных данных: 0.7538140356511965

Значение C: 13
F1_score на валидационных данных: 0.7535812007081925

Значение C: 14
F1_score на валидационных данных: 0.7533011272141708

Лучшее значение C: 7
Лучший F1_score на валид

Модель LGBMClassifier

In [13]:
model = LGBMClassifier(random_state=12345, class_weight='balanced')
parameters = {
    'max_depth': [5],
    'num_leaves': [10],
    'learning_rate': [0.1],
    'n_estimators' : [250],
}
lgbm_grid = GridSearchCV(model, parameters, scoring='f1', cv=3, n_jobs=-1)

In [14]:
lgbm_grid.fit(tf_idf, target_train)

GridSearchCV(cv=3,
             estimator=LGBMClassifier(class_weight='balanced',
                                      random_state=12345),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5],
                         'n_estimators': [250], 'num_leaves': [10]},
             scoring='f1')

In [15]:
lgbm_fit_score = lgbm_grid.best_score_
print(
    f'Лучший показатель F1: {lgbm_fit_score:.3f}'
)

Лучший показатель F1: 0.727


Тестирование

In [16]:
model = LogisticRegression(class_weight='balanced', 
                          random_state=12345, 
                          max_iter=100, 
                          solver='liblinear',
                          C=7)
model.fit(tf_idf, target_train)
predictions = model.predict(tf_idf_test)
lin_f1 = f1_score(target_test, predictions)
lin_conf_mat = confusion_matrix(target_test, predictions)

print(
    f'F1-score: {lin_f1:.3f}\n'
    f'Количество истинно отрицательных результатов: {lin_conf_mat[0][0]}\n'
    f'Количество ложноотрицательных результатов: {lin_conf_mat[1][0]}\n'
    f'Количество истинно положительных результатов: {lin_conf_mat[1][1]}\n'
    f'Количество ложноположительных результатов: {lin_conf_mat[0][1]}\n'
)

F1-score: 0.755
Количество истинно отрицательных результатов: 13720
Количество ложноотрицательных результатов: 272
Количество истинно положительных результатов: 1340
Количество ложноположительных результатов: 598



Выводы: 

Лучший результат показала модель логистической регрессии с параметрами С = 7, max_iter = 100 был достигнут целевой порог: F1-score выше 0.75, у этой модели наименьший показатель ложноотрицательных результатов - 272 (что является преимуществом для данной задачи).

## Выводы

В датафрейме data 159292 записей, названия столбцов адекватны содержащимся в них данным. Было проверено, нет ли дубликатов. Выявлено, что в датафрейме 143106 нетоксичных комментарией и 16186 токсичных. Текст был очищен и лемматизирован с помощью WordNetLemmatizer, был создан столбец с лемматизированным текстом. Были подготовлены тестовые и тренировочные выборки для обучения. Создана матрица cо значениями TF-IDF и указаны стоп-слова.

Лучший результат показала модель логистической регрессии с параметрами С = 7, max_iter = 100 был достигнут целевой порог: F1-score выше 0.75, у этой модели наименьший показатель ложноотрицательных результатов - 272 (что является преимуществом для данной задачи).

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны